In [ ]:
!pip install efficientnet

     |████████████████████████████████| 51kB 5.1MB/s 


In [ ]:
import tensorflow as tf
from efficientnet.keras import EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import Dropout, LeakyReLU, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
h, w, c = 192, 192, 3

input_shape = (h, w, c)

In [ ]:
encoder = EfficientNetB7(include_top=False, weights="imagenet", input_shape=(h, w ,3))

258441216/258434480 [==============================] - 7s 0us/step


In [ ]:
for i, l in enumerate(encoder.layers):
    print(i, l.output)

0 KerasTensor(type_spec=TensorSpec(shape=(None, 192, 192, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
1 KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 64), dtype=tf.float32, name=None), name='stem_conv/Conv2D:0', description="created by layer 'stem_conv'")
2 KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 64), dtype=tf.float32, name=None), name='stem_bn/FusedBatchNormV3:0', description="created by layer 'stem_bn'")
3 KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 64), dtype=tf.float32, name=None), name='stem_activation/mul:0', description="created by layer 'stem_activation'")
4 KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 64), dtype=tf.float32, name=None), name='block1a_dwconv/depthwise:0', description="created by layer 'block1a_dwconv'")
5 KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 64), dtype=tf.float32, name=None), name='block1a_bn/FusedBatchNormV3:0', description="created by layer 'block1a_b

In [ ]:
def reccurent_block_old(x, filters, a=0.1):
    x1 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same", kernel_regularizer=l2(2e-4))(x)
    x1 = LeakyReLU(alpha=a)(x1)

    x2 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same", kernel_regularizer=l2(2e-4))(x1)              
    x2 = LeakyReLU(alpha=a)(x2)
    x12 = Add()([x1, x2])

    x3 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same", kernel_regularizer=l2(2e-4))(x12) 
    x3 = LeakyReLU(alpha=a)(x3)
    x13 = Add()([x1, x3])

    x4 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same", kernel_regularizer=l2(2e-4))(x13)
    x4 = LeakyReLU(alpha=a)(x4)
    x14 = Add()([x1, x4])

    x = LeakyReLU(alpha=a)(x14)

    x = BatchNormalization(axis=3)(x)
    return x

In [ ]:
def Recurrrent_block(x, filters, a=0.01, dr=0.05):
    x1 = Conv_block(x, filters, a, dr)
    c1 = Concatenate(axis=-1)([x, x1])

    x2 = Conv_block(c1, filters, a, dr)
    c2 = Concatenate(axis=-1)([x, x2])

    x3 = Conv_block(c2, filters, a, dr)
    c3 = Concatenate(axis=-1)([x, x3])

    return c3

# Above Old

---

# Below New

In [ ]:
def Conv_block(x, filters, a=0.01, dr=0.05):
    y = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x)
    y = BatchNormalization(axis=-1)(y)
    y = LeakyReLU(alpha=a)(y)
    y = Dropout(rate=dr)(y)

    return y

In [ ]:
def Residual_block(x, filters, a=0.01, dr=0.05):
    y = Conv_block(x, filters, a, dr)
    y = Conv_block(y, filters, a, dr)
    y = Add(axis=-1)([x, y])

    return y

In [ ]:
def Dense_block(x, filters, a=0.01, dr=0.05):
    x1 = Conv_block(x, filters, a, dr)
    c1 = Concatenate(axis=-1)([x, x1])

    x2 = Conv_block(c1, filters, a, dr)
    c2 = Concatenate(axis=-1)([c1, x2])

    x3 = Conv_block(c2, filters, a, dr)
    c3 = Concatenate(axis=-1)([c2, x3])

    return c3

In [ ]:
def Dense_block(x, filters, a=0.01, dr=0.05, depth=2):

    for _ in range(depth):
        xn = Conv_block(x, filters, a, dr)
        x = Concatenate(axis=-1)([x, xn])

    return x

In [ ]:
# def R2_block(x, filters, a=0.01, dr=0.05, depth=3):
#     x1 = Conv_block(x, filters, a, dr)

#     xn = Conv_block(x1, filters, a, dr)
#     cn = Add(axis=-1)([x1, xn])

#     for _ in range(depth - 1):
#         xn = Conv_block(cn, filters, a, dr)
#         cn = Add(axis=-1)([x1, xn])

#     return cn

In [ ]:
def R2_block(x, filters, a=0.01, dr=0.05, depth=3):
    x1 = Conv_block(x, filters, a, dr)

    xn = Conv_block(x1, filters, a, dr)
    cn = Concatenate(axis=-1)([x1, xn])

    for _ in range(depth - 1):
        xn = Conv_block(cn, filters, a, dr)
        cn = Concatenate(axis=-1)([x1, xn])

    return cn

In [ ]:
print(encoder.layers[49].output)
print(encoder.layers[152].output)
print(encoder.layers[255].output)
print(encoder.layers[551].output)
print(encoder.layers[744].output)
print(encoder.layers[802].output)

KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 32), dtype=tf.float32, name=None), name='block1d_add/add:0', description="created by layer 'block1d_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 48), dtype=tf.float32, name=None), name='block2g_add/add:0', description="created by layer 'block2g_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 24, 24, 80), dtype=tf.float32, name=None), name='block3g_add/add:0', description="created by layer 'block3g_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 12, 12, 224), dtype=tf.float32, name=None), name='block5j_add/add:0', description="created by layer 'block5j_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 6, 6, 384), dtype=tf.float32, name=None), name='block6m_add/add:0', description="created by layer 'block6m_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 6, 6, 640), dtype=tf.float32, name=None), name='block7d_add/add:0', description="created by layer 'block7d_add'")


In [ ]:
def efficientB7R2Unet(input_shape):
    a = 0.01
    dr = 0.1
    depth = 5

    Encoder = EfficientNetB7(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 1
    input = Encoder.input
    en1 = R2_block(input, 32, a, dr, depth)

    en1 = Conv_block(en1, 32, a, dr)

    # contracting level 2
    en2 = Encoder.layers[49].output

    # contracting level 3
    en3 = Encoder.layers[152].output

    # contracting level 4
    en4 = Encoder.layers[255].output

    # middle level 5
    en5 = Encoder.layers[551].output
    cv5 = R2_block(en5, 224, a, dr, depth)
    up5 = Conv2DTranspose(224, (2, 2), strides=(2, 2), padding="same")(cv5)

    # expandsion level 4
    cv4 = Concatenate(axis=-1)([up5, en4])
    cv4 = R2_block(cv4, 80, a, dr, depth)
    up4 = Conv2DTranspose(80, (2, 2), strides=(2, 2), padding="same")(cv4)

    # expandsion level 3
    cv3 = Concatenate(axis=-1)([up4, en3])
    cv3 = R2_block(cv3, 48, a, dr, depth)
    up3 = Conv2DTranspose(48, (2, 2), strides=(2, 2), padding="same")(cv3)

    # expandsion level 2
    cv2 = Concatenate(axis=-1)([up3, en2])
    cv2 = R2_block(cv2, 32, a, dr, depth)
    up2 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(cv2)

    # expandsion level 1
    cv1 = Concatenate(axis=-1)([up2, en1])
    cv1 = R2_block(cv1, 32, a, dr, depth)

    cv1 = Conv_block(cv1, 32, a, dr)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv1)

    model = tf.keras.Model(inputs=[input], outputs=[out])

    model.compile(optimizer=Adam(0.003), loss="binary_crossentropy", metrics=["acc", "mse"])

    return model

In [ ]:
model = efficientB7R2Unet(input_shape)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 192, 192, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 96, 96, 64)   1728        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 96, 96, 64)   256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 96, 96, 64)   0           stem_bn[0][0]                    
______________________________________________________________________________________________